In [ ]:
# import libraries and data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras import optimizers, regularizers
import seaborn as sns
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from statistics import mean

# set random seed for more reproducible results
np.random.seed(1)

# read the data using pandas csv read

input_data_1 = '/content/drive/Shareddrives/ACS6403 - Group Project/Preprocessed Data/input_data.txt'  # Update the path
output_data_1 = '/content/drive/Shareddrives/ACS6403 - Group Project/Preprocessed Data/output_data.txt'  # Update the path

input_data = pd.read_csv(input_data_1, sep=',')
output_data = pd.read_csv(output_data_1, sep='\s+')

# convert pandas dataframe to a numpy array
Input_data_raw = np.array(input_data)
Output_data_raw = np.array(output_data)

print(Input_data_raw)
print(Output_data_raw)

# scaler = StandardScaler()           # define scaling function
scaler = MinMaxScaler()
Input = scaler.fit_transform(Input_data_raw)         # standardize input data
Output = scaler.fit_transform(Output_data_raw)         # standardize output data
num_input_features = Input.shape[1]           # number of features
num_output_features = Output.shape[1]            # number of classes

print(Input)
print(Output)

print(num_input_features)
print(num_output_features)

# # split the data set into training and validation sets
# X_train, X_val, Y_train, Y_val = train_test_split(Input, Output,
# test_size=0.25, random_state=2)

[[3.231500e+02 1.000000e-05 6.132245e-03]
 [3.231500e+02 1.000000e-05 1.225449e-02]
 [3.231500e+02 1.000000e-05 1.837673e-02]
 ...
 [9.731500e+02 2.000000e-01 2.877555e-01]
 [9.731500e+02 2.000000e-01 2.938778e-01]
 [9.731500e+02 2.000000e-01 3.000000e-01]]
[[-4.7338106e+04 -1.0599773e+04 -2.7131550e+04 ...  1.4517185e-25
   5.9826421e-25  1.3981116e-05]
 [-4.7308263e+04 -1.0616659e+04 -2.5240740e+04 ...  1.5486813e-25
   7.6536221e-25  1.3943969e-05]
 [-4.7278894e+04 -1.0633829e+04 -2.4122189e+04 ...  1.6521204e-25
   9.7692130e-25  1.3906912e-05]
 ...
 [-6.3567739e+04 -4.8424248e+04 -6.3107304e+04 ...  3.3099485e-11
   1.4352324e-10  1.5758037e-05]
 [-6.3601686e+04 -4.8488750e+04 -6.2971021e+04 ...  3.3810930e-11
   1.4741547e-10  1.5725362e-05]
 [-6.3635113e+04 -4.8554450e+04 -6.2837199e+04 ...  3.4537655e-11
   1.5129962e-10  1.5692779e-05]]
[[0.         0.         0.02040816]
 [0.         0.         0.04081633]
 [0.         0.         0.06122447]
 ...
 [1.         1.         0.959

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#3rd run best 0.05
model2 = Sequential()

model2.add(Dense(64, kernel_regularizer=regularizers.L2(l2=0.005), input_shape=(num_input_features,)))
model2.add(BatchNormalization())
model2.add(Activation("gelu"))

model2.add(Dense(48, kernel_regularizer=regularizers.L2(l2=0.005)))
model2.add(BatchNormalization())
model2.add(Activation("gelu"))

# model2.add(Dense(32, kernel_regularizer=regularizers.L2(l2=0.005)))
# model2.add(BatchNormalization())
# model2.add(Activation("gelu"))

model2.add(Dense(16, kernel_regularizer=regularizers.L2(l2=0.005)))
model2.add(BatchNormalization())
model2.add(Activation("gelu"))

model2.add(Dropout(0.2))
model2.add(Dense(num_output_features))

# optimization options
epsilon = 0.001
opt = optimizers.Adam(learning_rate=epsilon)

# compile the model with the optimization options
model2.compile(loss="mean_absolute_error", optimizer=opt, metrics=["accuracy"])

model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 64)                256       
                                                                 
 batch_normalization_6 (Bat  (None, 64)                256       
 chNormalization)                                                
                                                                 
 activation_6 (Activation)   (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 48)                3120      
                                                                 
 batch_normalization_7 (Bat  (None, 48)                192       
 chNormalization)                                                
                                                                 
 activation_7 (Activation)   (None, 48)               

In [ ]:
# MinMaxScaler
early_stopping = EarlyStopping(monitor='val_accuracy',patience=10, restore_best_weights=True)
# fit the keras model on the dataset store loss and accuracy in history
history2 = model2.fit(X_train, Y_train, epochs=50, batch_size=128, validation_data=(X_val, Y_val), callbacks=[early_stopping])
predictions = model2.predict(X_val)
mae = mean_absolute_error(Y_val, predictions, multioutput='raw_values')
print(mae)
print(mean(mae))

Epoch 1/50
440/440 [==============================] - 6s 8ms/step - loss: 0.4287 - accuracy: 0.4099 - val_loss: 0.1846 - val_accuracy: 0.5586
Epoch 2/50
440/440 [==============================] - 4s 9ms/step - loss: 0.1251 - accuracy: 0.5330 - val_loss: 0.0813 - val_accuracy: 0.6139
Epoch 3/50
440/440 [==============================] - 3s 6ms/step - loss: 0.0660 - accuracy: 0.6015 - val_loss: 0.0516 - val_accuracy: 0.7834
Epoch 4/50
440/440 [==============================] - 3s 6ms/step - loss: 0.0435 - accuracy: 0.7027 - val_loss: 0.0380 - val_accuracy: 0.8346
Epoch 5/50
440/440 [==============================] - 3s 6ms/step - loss: 0.0341 - accuracy: 0.7695 - val_loss: 0.0387 - val_accuracy: 0.8219
Epoch 6/50
440/440 [==============================] - 4s 8ms/step - loss: 0.0312 - accuracy: 0.8066 - val_loss: 0.0193 - val_accuracy: 0.9214
Epoch 7/50
440/440 [==============================] - 3s 8ms/step - loss: 0.0298 - accuracy: 0.8163 - val_loss: 0.0224 - val_accuracy: 0.8796
Epoch 

In [ ]:
num_folds = 5

KF = KFold(n_splits=num_folds, shuffle=False)

In [ ]:
mae_scores = []
early_stopping = EarlyStopping(monitor='val_accuracy',patience=10, restore_best_weights=True)
for train_index, test_index in KF.split(Input):
  X_train, X_val = Input[train_index], Input[test_index]
  Y_train, Y_val = Output[train_index], Output[test_index]
  print(X_train)
  print('***************')
  print(X_val)
  print('---------------')
  print(Y_train)
  print('***************')
  print(Y_val)
  print('===============')
  history2 = model2.fit(X_train, Y_train, epochs=50, batch_size=128, validation_data=(X_val, Y_val))#, callbacks=[early_stopping])
  predictions = model2.predict(X_val)
  mae = mean_absolute_error(Y_val, predictions, multioutput='raw_values')
  mae_scores.append(mae)
print(mae_scores)

[[0.20689662 0.         0.02040816]
 [0.20689662 0.         0.04081633]
 [0.20689662 0.         0.06122447]
 ...
 [1.         1.         0.95918364]
 [1.         1.         0.97959199]
 [1.         1.         1.        ]]
***************
[[0.         0.         0.02040816]
 [0.         0.         0.04081633]
 [0.         0.         0.06122447]
 ...
 [0.17241385 1.         0.97959199]
 [0.17241385 1.         1.        ]
 [0.20689662 0.         0.        ]]
---------------
[[6.29403517e-01 8.59997586e-01 8.24688760e-01 ... 3.87531467e-09
  1.88409761e-09 3.60568697e-01]
 [6.29434515e-01 8.59378965e-01 8.45341102e-01 ... 4.05598241e-09
  2.23389253e-09 3.53224477e-01]
 [6.29464710e-01 8.58757103e-01 8.57416345e-01 ... 4.24507255e-09
  2.64440597e-09 3.45898109e-01]
 ...
 [6.36063001e-01 3.42995869e-03 6.28259952e-01 ... 9.58359362e-01
  5.20326367e-01 6.78475513e-01]
 [6.35787894e-01 1.73075902e-03 6.29329621e-01 ... 9.78958473e-01
  5.34437182e-01 6.72065480e-01]
 [6.35517001e-01 0.00000

In [ ]:
from joblib import dump, load

mae_scores = []
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
fold_number = 0

early_stopping = EarlyStopping(monitor='val_accuracy',patience=10, restore_best_weights=True)
for train_index, test_index in KF.split(Input):
  X_train, X_val = Input[train_index], Input[test_index]
  Y_train, Y_val = Output[train_index], Output[test_index]
  print(X_train)
  print('***************')
  print(X_val)
  print('---------------')
  print(Y_train)
  print('***************')
  print(Y_val)
  print('===============')
  history2 = model2.fit(X_train, Y_train, epochs=50, batch_size=128, validation_data=(X_val, Y_val), callbacks=[early_stopping])

  # Save the model at the end of training for each fold
  model_filename = f'model_fold_{fold_number}.h5'
  model2.save(model_filename)
  print(f'Model saved for fold {fold_number} at {model_filename}')


  predictions = model2.predict(X_val)
  mae = mean_absolute_error(Y_val, predictions, multioutput='raw_values')
  mae_scores.append(mae)
print(mae_scores)

[[0.20689662 0.         0.02040816]
 [0.20689662 0.         0.04081633]
 [0.20689662 0.         0.06122447]
 ...
 [1.         1.         0.95918364]
 [1.         1.         0.97959199]
 [1.         1.         1.        ]]
***************
[[0.         0.         0.02040816]
 [0.         0.         0.04081633]
 [0.         0.         0.06122447]
 ...
 [0.17241385 1.         0.97959199]
 [0.17241385 1.         1.        ]
 [0.20689662 0.         0.        ]]
---------------
[[6.29403517e-01 8.59997586e-01 8.24688760e-01 ... 3.87531467e-09
  1.88409761e-09 3.60568697e-01]
 [6.29434515e-01 8.59378965e-01 8.45341102e-01 ... 4.05598241e-09
  2.23389253e-09 3.53224477e-01]
 [6.29464710e-01 8.58757103e-01 8.57416345e-01 ... 4.24507255e-09
  2.64440597e-09 3.45898109e-01]
 ...
 [6.36063001e-01 3.42995869e-03 6.28259952e-01 ... 9.58359362e-01
  5.20326367e-01 6.78475513e-01]
 [6.35787894e-01 1.73075902e-03 6.29329621e-01 ... 9.78958473e-01
  5.34437182e-01 6.72065480e-01]
 [6.35517001e-01 0.00000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


469/469 [==============================] - 1s 2ms/step
[[0.         0.         0.02040816]
 [0.         0.         0.04081633]
 [0.         0.         0.06122447]
 ...
 [1.         1.         0.95918364]
 [1.         1.         0.97959199]
 [1.         1.         1.        ]]
***************
[[0.20689662 0.         0.02040816]
 [0.20689662 0.         0.04081633]
 [0.20689662 0.         0.06122447]
 ...
 [0.37931031 1.         0.97959199]
 [0.37931031 1.         1.        ]
 [0.41379308 0.         0.        ]]
---------------
[[7.67588057e-01 9.99853873e-01 9.10629325e-01 ... 1.85743647e-16
  4.77462260e-16 3.29887250e-01]
 [7.67829905e-01 9.99409039e-01 9.25470066e-01 ... 4.66488822e-16
  1.08325610e-15 3.22599921e-01]
 [7.68067912e-01 9.98956724e-01 9.34249438e-01 ... 7.65985415e-16
  1.85023832e-15 3.15330248e-01]
 ...
 [6.36063001e-01 3.42995869e-03 6.28259952e-01 ... 9.58359362e-01
  5.20326367e-01 6.78475513e-01]
 [6.35787894e-01 1.73075902e-03 6.29329621e-01 ... 9.78958473e-01
  

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


469/469 [==============================] - 1s 2ms/step
[[0.         0.         0.02040816]
 [0.         0.         0.04081633]
 [0.         0.         0.06122447]
 ...
 [1.         1.         0.95918364]
 [1.         1.         0.97959199]
 [1.         1.         1.        ]]
***************
[[0.41379308 0.         0.02040816]
 [0.41379308 0.         0.04081633]
 [0.41379308 0.         0.06122447]
 ...
 [0.58620692 1.         0.97959199]
 [0.58620692 1.         1.        ]
 [0.62068969 0.         0.        ]]
---------------
[[7.67588057e-01 9.99853873e-01 9.10629325e-01 ... 1.85743647e-16
  4.77462260e-16 3.29887250e-01]
 [7.67829905e-01 9.99409039e-01 9.25470066e-01 ... 4.66488822e-16
  1.08325610e-15 3.22599921e-01]
 [7.68067912e-01 9.98956724e-01 9.34249438e-01 ... 7.65985415e-16
  1.85023832e-15 3.15330248e-01]
 ...
 [6.36063001e-01 3.42995869e-03 6.28259952e-01 ... 9.58359362e-01
  5.20326367e-01 6.78475513e-01]
 [6.35787894e-01 1.73075902e-03 6.29329621e-01 ... 9.78958473e-01
  

...

469/469 [==============================] - 1s 3ms/step
[[0.         0.         0.02040816]
 [0.         0.         0.04081633]
 [0.         0.         0.06122447]
 ...
 [1.         1.         0.95918364]
 [1.         1.         0.97959199]
 [1.         1.         1.        ]]
***************
[[0.62068969 0.         0.02040816]
 [0.62068969 0.         0.04081633]
 [0.62068969 0.         0.06122447]
 ...
 [0.79310338 1.         0.97959199]
 [0.79310338 1.         1.        ]
 [0.82758615 0.         0.        ]]
---------------
[[7.67588057e-01 9.99853873e-01 9.10629325e-01 ... 1.85743647e-16
  4.77462260e-16 3.29887250e-01]
 [7.67829905e-01 9.99409039e-01 9.25470066e-01 ... 4.66488822e-16
  1.08325610e-15 3.22599921e-01]
 [7.68067912e-01 9.98956724e-01 9.34249438e-01 ... 7.65985415e-16
  1.85023832e-15 3.15330248e-01]
 ...
 [6.36063001e-01 3.42995869e-03 6.28259952e-01 ... 9.58359362e-01
  5.20326367e-01 6.78475513e-01]
 [6.35787894e-01 1.73075902e-03 6.29329621e-01 ... 9.78958473e-01
  

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


469/469 [==============================] - 1s 2ms/step
[[0.         0.         0.02040816]
 [0.         0.         0.04081633]
 [0.         0.         0.06122447]
 ...
 [0.79310338 1.         0.97959199]
 [0.79310338 1.         1.        ]
 [0.82758615 0.         0.        ]]
***************
[[0.82758615 0.         0.02040816]
 [0.82758615 0.         0.04081633]
 [0.82758615 0.         0.06122447]
 ...
 [1.         1.         0.95918364]
 [1.         1.         0.97959199]
 [1.         1.         1.        ]]
---------------
[[7.67588057e-01 9.99853873e-01 9.10629325e-01 ... 1.85743647e-16
  4.77462260e-16 3.29887250e-01]
 [7.67829905e-01 9.99409039e-01 9.25470066e-01 ... 4.66488822e-16
  1.08325610e-15 3.22599921e-01]
 [7.68067912e-01 9.98956724e-01 9.34249438e-01 ... 7.65985415e-16
  1.85023832e-15 3.15330248e-01]
 ...
 [7.24390581e-01 2.35548531e-01 7.17725261e-01 ... 9.63018190e-02
  6.84704349e-02 6.26204959e-01]
 [7.24196490e-01 2.33993140e-01 7.18636994e-01 ... 9.87125675e-02
  

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


469/469 [==============================] - 1s 2ms/step
[array([0.01495947, 0.03231525, 0.01761941, 0.01430947, 0.00403356,
       0.00846814, 0.03749405]), array([0.01221762, 0.02247306, 0.01188413, 0.01136099, 0.00328906,
       0.00230588, 0.01850969]), array([0.03870645, 0.08503097, 0.03848143, 0.00861983, 0.00230311,
       0.00188614, 0.01472042]), array([0.03221133, 0.04818908, 0.03200253, 0.01368371, 0.00494441,
       0.00498527, 0.02206266]), array([0.0578315 , 0.12497769, 0.03907978, 0.25605412, 0.14548732,
       0.09716027, 0.01399624])]
